In [8]:
using Gen, PyCall, Plots
import Random, Logging

Logging.disable_logging(Logging.Info);
Random.seed!(42)


Random.TaskLocalRNG()

In [9]:
#dataset
function make_dataset_line(n)
    # true parameters
    prob_outlier = 0.2
    true_inlier_noise = 0.5
    true_outlier_noise = 5.0
    
    true_slope = randn()*2
    true_intercept = randn()*5
    #print the true parameters
    println("True slope: ", true_slope)
    println("True intercept: ", true_intercept)    

    xs = collect(range(-5, stop=5, length=n))
    ys = Float64[]
    for (i, x) in enumerate(xs)
        if randn() < prob_outlier
            y = randn() * true_outlier_noise
        else
            y = true_slope * x + true_intercept + randn() * true_inlier_noise
        end
        push!(ys, y)
    end
    (xs, ys)
end

make_dataset (generic function with 1 method)

In [10]:
#model, adapted from playground to only use normal distributions
@gen function regression_line(xs::Vector{<:Real})
    # ...

    slope ~ normal(0, 2)
    intercept ~ normal(2, 2)

    noise ~ normal(0.2, 0.03)
    prob_outlier ~ normal(0, 1)

    # Next, we generate the actual y coordinates.
    n = length(xs)
    ys = Float64[]

    for i = 1:n
        # Decide whether this point is an outlier, and set
        # mean and standard deviation accordingly
        outlier= {:data => (i, :is_outlier)} ~ normal(prob_outlier,0.5)

        if outlier > 0
            (mu, std) = (0., 10.)
        else
            (mu, std) = (xs[i] * slope + intercept, noise)
        end
        # Sample a y value for this point

        y={:data => i => :y} ~ normal(mu, std)

        push!(ys, y)

    end
    ys
end;

In [11]:
@gen function testProposal_line(trace)
    
    slope ~ normal(trace[:slope], 0.1)
    intercept ~ normal(trace[:intercept], 0.1)
    noise ~ normal(trace[:noise], 0.01)
    prob_outlier ~ normal(trace[:prob_outlier], 0.01)

    for i =1:10
        outlier=@trace(normal(prob_outlier,0.5),(:data => i => :is_outlier))
        if outlier > 0
            (mu, std) = (0., 10.)
        else
            (mu, std) = (xs[i] * slope + intercept, noise)
        end
        # Sample a y value for this point
        y=@trace(normal(mu, std), (:data,i,:y))
    end
    return nothing
end;

In [12]:
#nn model line
#inputs are:                                values
# - x values                                10    
# - y values                                10
# - address of sample (one hot encoded)     5
# - instance id (one hot encoded)           10

# - if lstm: previous hidden state

#outputs are:
# - mean
# - std

tf = pyimport("tensorflow")
keras = pyimport("keras")

def nn_line():
    nn_model = keras.models.Sequential()
    nn_model.add(keras.layers.Dense(10, input_shape=(35,), activation="relu"))
    nn_model.add(keras.layers.Dense(10, activation="relu"))
    nn_model.add(keras.layers.Dense(2, activation="linear"))

    #compile the model
    nn_model.compile(loss="mse", optimizer="adam")

    #train the model 

    #predict the mean and std
    test=rand(1,35)
    mean, std = nn_model.predict(test)
    return nn_model

1/1 [==============================] - 0s 37ms/step


1×2 Matrix{Float32}:
 0.097332  0.228106

In [19]:
# main 

xs, ys = make_dataset(10)

constraints = choicemap()
for (i, y) in enumerate(ys)
    constraints[:data => i => :y] = y
end

(trace, _) = Gen.generate(regression, (xs,), constraints)

# include a proposal
for i=1:10
    (trace, _) = Gen.mh(trace, testProposal, ())
end

println(trace)

True slope: 2.246364846644055
True intercept: 0.18557850595062747
Gen.DynamicDSLChoiceMap(Trie{Any, Gen.ChoiceOrCallRecord}(Dict{Any, Gen.ChoiceOrCallRecord}(:intercept => Gen.ChoiceOrCallRecord{Float64}(4.2467773513471325, -2.2430867720804226, NaN, true), :slope => Gen.ChoiceOrCallRecord{Float64}(0.4953908430305271, -1.64276222468443, NaN, true), :prob_outlier => Gen.ChoiceOrCallRecord{Float64}(1.0936093038452734, -1.5169291879331444, NaN, true), :noise => Gen.ChoiceOrCallRecord{Float64}(0.22690823024455312, 2.185367778063152, NaN, true)), Dict{Any, Trie{Any, Gen.ChoiceOrCallRecord}}(:data => Trie{Any, Gen.ChoiceOrCallRecord}(Dict{Any, Gen.ChoiceOrCallRecord}((10, :is_outlier) => Gen.ChoiceOrCallRecord{Float64}(1.1671055150129417, -0.23659473875673231, NaN, true), (9, :is_outlier) => Gen.ChoiceOrCallRecord{Float64}(1.4522226525731412, -0.4829984204163581, NaN, true), (8, :is_outlier) => Gen.ChoiceOrCallRecord{Float64}(1.5495659267518567, -0.6415842365894796, NaN, true), (3, :is_outlie

LoadError: Did not visit all constraints

In [87]:
@gen function coin_model()
    val1 ~ normal(0, 1)
    if val1 > 0
        val2 ~ normal(0.5, 0.5)
    else
        val2 ~ normal(-0.5, 0.5)
    end
end;

In [88]:
@gen function coin_proposal(trace)
    val1 ~ normal(trace[:val1], 0.1)
end;

In [89]:
function block_resimulation_inference_coin(val2)
    observations = Gen.choicemap()
    observations[:val2] = val2
    (tr, ) = Gen.generate(coin_model, (), observations)
    for iter=1:10000
        (tr, ) = mh(tr, coin_proposal, ())
        tr
    end
    tr
end;

In [90]:
trace = Gen.simulate(coin_model, ());
choices=Gen.get_choices(trace)
val2=choices[:val2]
println("True value")
println(choices[:val1])
println(choices[:val2])

tr = block_resimulation_inference(val2)
choices=Gen.get_choices(tr)

println("Inferred value")
println(choices[:val1])
println(choices[:val2])

True value
0.30708652879950665
0.4414324065042088
Inferred value
-0.016143020838595937
0.4414324065042088


In [178]:
@gen function burglar_model(chance)
    burglar ~ bernoulli(chance)

    if burglar
        alarm ~ bernoulli(0.9)
    else
        alarm ~ bernoulli(0.15)
    end

    if alarm
        john ~ bernoulli(0.6)
        mary ~ bernoulli(0.7)
    else
        john ~ bernoulli(0.3)
        mary ~ bernoulli(0.1)
    end

end;

In [182]:
torch = pyimport("torch")
nn = torch.nn
F = nn.functional

function get_nn():

    # inputs: observe = [john, mary], addresses (one hot), x_t-1

    model = nn.Sequential(
        nn.Linear(5, 10),
        nn.ReLU(),
        nn.Linear(10, 10),
        nn.ReLU(),
        nn.Linear(10, 1)
    )
    return model
end

function loss(x,y)
    
end

function train(model, inputs, outputs, epochs)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch=1:epochs
        optimizer.zero_grad()

        outputs_pred = model(inputs)
        loss = criterion(outputs_pred, outputs)
        
        loss.backward()
        optimizer.step()
    end
end

LoadError: syntax: newline not allowed after ":" used for quoting

In [179]:
@gen function burglar_proposal(trace)
    burglar ~ bernoulli(0.6)
end;

In [180]:
function block_resimulation_burglar(john,mary)
    observations = Gen.choicemap()
    observations[:john] = john
    observations[:mary] = mary
    (tr, ) = Gen.generate(burglar_model, (0.2,), observations)
    for iter=1:10000
        (tr, ) = mh(tr, burglar_proposal, ())
        (tr, ) = mh(tr, select(:alarm))
    end
    tr
end;

In [181]:
trace = Gen.simulate(burglar_model, (0.6,));

println("True value")
c=make_constraints_burglar(trace)
println(c)

tr = block_resimulation_burglar(c[:john],c[:mary])
choices=Gen.get_choices(tr)

println("Inferred value")
c=make_constraints_burglar(tr)
c

True value
DynamicChoiceMap(Dict{Any, Any}(:burglar => false, :alarm => false, :mary => false, :john => true), Dict{Any, Any}())
Inferred value


│
├── :burglar : true
│
├── :alarm : false
│
├── :mary : false
│
└── :john : true
